In [1]:
import csv
import pandas as pd
import Levenshtein

In [2]:
# Read input CSV file
input_file = 'sp_csv.csv'
output_file = 'output.tsv'

In [3]:
# Read CSV file into a DataFrame
df = pd.read_csv(input_file, encoding='utf-8')

In [4]:
# Dictionary to store counts of each similar string
similar_counts = {}

# Calculate Levenshtein distances
for i, row in df.iterrows():
    current_id = row['xml:id']
    current_count = row['count']
    current_string = current_id.split(',')[0]

    # Check Levenshtein distance with all other strings
    for j, other_row in df.iterrows():
        other_id = other_row['xml:id']
        other_count = other_row['count']
        other_string = other_id.split(',')[0]

        distance = Levenshtein.distance(current_string, other_string)
        if 0 < distance <= 2:
            similar_counts.setdefault(current_string, {}).setdefault(other_string, 0)
            similar_counts[current_string][other_string] += other_count

# Find the correct corresponding string based on the most occurrences
correct_strings = {}
for string, similar_strings in similar_counts.items():
    most_common = max(similar_strings.items(), key=lambda x: x[1])
    if similar_strings:  # Check if there are any similar strings
        if most_common[1] >= df.loc[df['xml:id'].str.startswith(string), 'count'].sum():
            correct_strings[string] = most_common[0]
        else:
            correct_strings[string] = string
    else:
        correct_strings[string] = string  

In [6]:
with open(output_file, 'w', newline='', encoding='utf-8') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(['xml:id', 'count', 'correct_string'])
    for row in df.itertuples(index=False):
        current_id = row[0]
        current_count = row[1]
        current_string = current_id.split(',')[0]
        correct_string = correct_strings.get(current_string, current_string)
        writer.writerow([current_id, current_count, correct_string])